In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 22 17:12:01 2021

@author: User
"""
# import 爬蟲所需要的東西
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import numpy as np
import os,sys,time
from PIL import Image
from urllib import request
import requests
import json
import sys
import modin.pandas as pd
from iteration_utilities import duplicates
import asyncio
import unittest
from numba import jit
import csv
import copy
import networkx as nx


# P.S.
# python 預設調用  print， 會調用 __str__ 函數。
# __rerp__ 是預覽



# 實作　Node 單位


class Node:
    #建構子，初始化節點
    #初始authority跟 hub,值皆為1
    def __init__(self, ID): #一定會有self這個值，並用self代稱上面的class_name(Node)
        self.ID = ID
        self.parents = []
        self.children = []  #初始的children 跟 parents串列裡面都是空的。(因為都沒有建立任何連結)
        self.auth = 1.0
        self.hub = 1.0
    # __str__ 是print會調用的函數
    def __str__(self):
        return str(self.ID)
        
    def show(self): #所有的function裡面都要有self，代指使用自己方法的本身。
        print(f"This's {self.ID}")
        
    #link_child 跟 link_parent，為網站間的有向連結，但其並沒有考慮到各child節點或各parent節點之間加入到"children"或"parent"串列的先後順序。
    #只能計算此節點所有連出的節點個數計算。
        
    def link_child(self,new_child): #這Method中有兩個參數，self如上述，"new child" 為其他的Node類物件，近來這個程式的代稱為"new_child"
        for child in self.children: #在呼叫Method link_child的 Node 中，依序搜尋children 串列。
            if(new_child.ID == child.ID): #若已經有了，回傳None
                print("duplicate") #若有重複印出"duplicate"
                #print(child.ID)
                return None
        self.children.append(new_child)
        
    def link_parent(self,new_parent): #同上
        for parent in self.parents:
            if (new_parent.ID == self.ID):
                print("duplicate")
                #print(self.ID)
                return None
        self.parents.append(new_parent)
        
    def show_children(self): #印出一個節點的所有孩子(印出這個node所有連出的連結)
        if len(self.children) == 0:
            print("this node :",self.ID,", children_list is empty.")
        else:
            for i in range(len(self.children)):
                print(self.children[i].ID)
        
    def show_parents(self): #印出一個節點的所有parent(印出這個node所有連入的連結)
        for i in range(len(self.parents)):
            print(self.parents[i].ID)
            
    def update_hub(self):  #更新此點的hub分數
        hub_score = 1
        for i in self.children:
            hub_score += i.hub
        return hub_score
    
    def update_auth(self):   #更新此點的authority分數
        authority_score = 0
        for i in self.parents:
            authority_score += i.auth
        return authority_score
    
    def show_indegree(self):  #算出所有此點的 in_degree
        pass
    
    def show_outdegree(self):  #算出所有此點的 out_degree
        pass
    
    def normalize_hub_score(self):   #將此點的hub分數除以全部的hub分數總合
        pass
    
    def normalize_authority_socre(self):   #將此點的authority分數除已全部的authority分數總合。
        pass
    
#實作 Graph

class Graph:   #在這個網絡上(圖)的所有網站(點)集合
    def __init__(self, G_name):
        self.name = G_name  #圖的名字
        self.nodes = [] #一開始的nodes集合為空
        self.edges = [] #所有的邊集合我也設一個串列存起來，但這邊我目前還沒有想到怎麼加入可以加入有向邊。
    
    def insert_node(self, node):  #將外來的點加入這個圖的點集合中
        try:    
            self.nodes.append(node)
            # print("successfully insert！")
        except error as e:
            print(f"insert fail:{e}")
    
    def display_hub_authority_score(self):
        pass
    
    def find_node(self, ID):  # 要找的點的名字，也就是網址
        for node in self.nodes: # 在自己圖中的節點做尋找
            if node.ID == ID:
                # print("found")
                return node # 找到的話就回傳節點
        else: # 否則就說沒找到
            # print("Not found")
            return None # 回傳空值

# 將所有 ID 一樣的都刪除。
def set_node_list(list_a):
    # 比較所有的ID名稱，先用for迴圈將所有的ID名稱存起來。
    all_name = [] #所有名字的列表
    for i in list_a:
        all_name.append(i.ID)
    all_name_count = [] #計算每個網站重複的數量
    for index,i in enumerate(list_a):  #對每個list_a 的元素做檢查
        count = 0
        for k in all_name: # 在所有的名字中搜尋，如果有一樣的名字，count就加一。
            if i.ID == k:
                count +=1
        all_name_count.append(count)
    for index,i in enumerate(all_name_count):  #超過一次以上的就刪除
        if i > 1:
            del list_a[index]
            del all_name[index]
            del all_name_count[index]
    return list_a                   #回傳一list 其為不重複的 node版本

def search_all_link(query_link):
    a = []  # 這個網站所有的 outlink
    html = requests.get(query_link, timeout = 10)
    # 為防止爬蟲卡死情況(request 不會自行斷開)，故上面設置 timeout 參數，設定固定時間若未完成則 斷開連結。
    html.encoding = 'UTF-8' #使用html編碼。
    sp = BeautifulSoup(html.text, 'html5lib') #指定'html5lib'作為解析器
    #若只使用sp.link 則只會找出第一筆
    #故這邊要使用"find.all"語法
    link_search = sp.find_all("a") #找出所有的 <a> (超連結)
    for i in range(len(link_search)):
        k = str(link_search[i].get("href"))  #這行一定要記得要轉type！！！轉成str不然有時候會無法切片取值。
        if k[0:5] == "https": #只留下"https"
            a.append(k)
    return a  #回傳一列網址(type list)

#先將所有node_list中的資料轉換成type str存入到list再寫入檔案
def node_to_text(all_node_list):
    node_list_txt = [] # node_list 的所有內容
    for i in all_node_list:
        tmp_list = [] # 初始化
        # 依序記錄結構化的node的每個資料
        tmp_list.append(i.ID)
        # 這邊嘗試寫遞迴函式，不停往下找
        # 加入cildren
        tmp_children = []
        for j in i.children:
            tmp_children.append(j.ID)
        tmp_list.append(tmp_children)
        # 加入parents
        tmp_parents = []
        for k in i.parents:
            tmp_parents.append(k.ID)
        tmp_list.append(tmp_parents)
        tmp_list.append(i.auth)
        tmp_list.append(i.hub)
        # 再加入list
        node_list_txt.append(tmp_list)
    return node_list_txt  # 回傳一個 list
def listDups(tmp_list_aaa): # 記錄重複的功能
    return list(duplicates(tmp_list_aaa))
def str_to_url(aaa): # aaa 為文字格式。
    bbb = aaa.split(",") # 利用逗號做切分 # bbb為list格式 # 會回傳一個list
    flag_head = False # 一開始起頭開關為False
    tmp_word_list = [] # 裝暫存要轉變成文字的網址，每次只會存一個網址，之後就會被清空
    tmp_url_list = [] # 裝整理好的網址
    for url in bbb: # 在這bbb串列中做遞迴
        for char in url:
            if char == "'": # 當遇見"單引號"時switch (此時Flag_head只有兩種狀態)
                flag_head = not(flag_head) # 轉換開關
                # tmp_word_list.append(char) # 將其收錄至暫存的字串列。
            if flag_head == True:
                tmp_word_list.append(char) # 因為只有h會將其打開，所以一路收錄char 至 tmp_word_list
            if flag_head == False:
                StrA = "".join(tmp_word_list)
                StrA = StrA[1:]  # 去掉前端的單引號。 
                if len(tmp_word_list) != 0: # 不為空才加入
                    tmp_url_list.append(StrA)
                tmp_word_list = [] # 清空tmp_word_list
    # 所以我只要再把這些網址NODE化之後再加入CHILDREN串列就好 =)
    return tmp_url_list #回傳整理好的網址

# 111/1/1發現這邊有問題，雖然建立的nodes名字都一樣，但是因為是從不同檔案load進來的，會有問題。
# 111/1/1目前解決方法，隨然load進來的是不同檔案，但是會load到同個圖中。
def load_node_from_csv_out_side(start_num, end_num, df_name): # 讀csv檔的資料 # 並且加入子節點 # 也同時加入父節點
    # 參數分別為，load加入的檔案的第幾筆到第幾筆，讀入黨案名稱。
# 每個點依序建立連結
    # 在這邊才讀取檔案是為了要避免讀取全域變數
    df = pd.read_csv(df_name)
    start_time = time.time()  #開始執行時間
    for i in range(start_num, end_num + 1): # 先存1-9000筆
        bbb = df["2"][i] # 將一整串str去頭去尾。
        url_list = str_to_url(bbb)  # 會回傳一個整理好的串列
        tmp2 = Graph_all.nodes[i] # 選取第幾個node
        for j in url_list: # 對找到的每個網站
            # 要先觀察 "j" 這個網站名字，是不是已經存在在原有的193721筆資料中。
            a = Graph_all.find_node(j)
            if a != None: # 有找到值，代表此網站已經在圖中了。此時 a 就已經是圖中的節點。 type == node
                tmp2.link_child(a)
                a.link_parent(tmp2)
            else: # 沒找到值，代表這是新的點
                tmp_node = Node(j) # 先將網站節點化
                Graph_all.insert_node(tmp_node) # 圖中也要加入這個新的點。
                Graph_all.nodes[i].link_child(tmp_node) # 加入node化的子節點。 # 這邊確認沒有重複。
                tmp_node.link_parent(tmp2) # 連結原節點。
    print("花費:" + str(float(time.time() - start_time)) + "秒")
    print("finish")
def Diff(li1, li2): 
    return (list(set(li1).symmetric_difference(set(li2))))  
# 判別兩列表不同元素
def diff(list_a, list_b):
    list_1 = list_a.copy()
    list_2 = list_b.copy()
    intersection = [x for x in list_1 for y in list_2 if x == y]
    return intersection

def load_out_link(start, end, csv_file_name): # 讀所有out_link # 回傳一列表
    
    df = pd.read_csv(csv_file_name)
    return_list = [] # 要回傳的列表
    for index in range(start,end+1): # 將前9000筆資料加入列表中
        bbb = df["2"][index]
        url_list = str_to_url(bbb)
        for j in url_list:
            if j[0:4] == "http": # 等於http的才加入。
                #print(j) # 撿查
                return_list.append(j) # 依序加入
    return return_list
    
# 實行二分搜尋
# 定義：在排序好的數列裡，找某 target，找到就回傳其 index，否則回傳 -1
def binary_search(data_list, key_data):  #要加入的圖的點的列表(已知且已排序), 要尋找的 資料
    #設置選取範圍的指標
    low = 0
    upper = len(data_list) - 1
    while low <= upper:
        mid = int((low + upper) / 2) #取中間索引的值
        if data_list[mid] < key_data:    #若搜尋值比中間的值大，將中間索引+1，取右半
            low = mid + 1
        elif data_list[mid] > key_data:  #若搜尋值比中間的值小，將中間索引+1，取左半
            upper = mid - 1
        else:                    #若搜尋值等於中間的值，則回傳
            return mid # 回傳的是索引值。
    return "Node Not Found!"


#  這邊一定要針對外部的圖進行修改，無法不用全域變數。
#  這邊其實已經解決先後順序的問題了，所以可以嘗試看看非同步的寫法。
#  此函式適用於所有load進來的檔案中已經沒有新node(所有的不重複網站名稱都已經存在在以建立之圖中)
#  之後應該要建立一另外之函式for 新加入的網站(以上述的csv檔案儲存)
def build_module_all_nodes_found(original_url, graph, out_link_list_http): 
    # 確定全部的點都加入了才可以使用此函式
    # 參數 原始的網站，要加入的圖(裡面的點已排序)(對這個圖進行操作)，out_link列表
        # 找到原始網站的那個點
        # 0.22s
        # 這邊改用 binary search
    All_graph_nodes_list = graph.nodes # 使用那個圖的點 #  指的是同一個位置 # 這個圖已經排序了
    key_1 = binary_search(All_graph_nodes_list, original_url)
    node_tmp = All_graph_nodes_list[key_1] 
   # print("網站",node_tmp.ID)
    for out_url in out_link_list_http:
        if len(out_link_list_http) != 0:
            key_2 = binary_search(All_graph_nodes_list, out_url)
            # 0.22s
            if key_2 != "Node Not Found!":
                # 利用指標操作找到那個點
                node_tmp_2 = All_graph_nodes_list[key_2] 
                node_tmp.link_child(node_tmp_2)
                node_tmp_2.link_parent(node_tmp)
             #   print("out_link", node_tmp_2.ID)
        else:
            print("empty_outlink")
def binary_search_for_nx(data_list, key_data):  #要加入的圖的點的列表(已知且已排序), 要尋找的 資料
    #設置選取範圍的指標
    low = 0
    upper = len(data_list) - 1
    while low <= upper:
        mid = int((low + upper) / 2) #取中間索引的值
        if data_list[mid] < key_data:    #若搜尋值比中間的值大，將中間索引+1，取右半
            low = mid + 1
        elif data_list[mid] > key_data:  #若搜尋值比中間的值小，將中間索引+1，取左半
            upper = mid - 1
        else:                    #若搜尋值等於中間的值，則回傳
            return mid # 回傳的是索引值。
    return "Node Not Found!"

def top_n_pagerank_for_nx(p, graph, n):
    L = sorted(p.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        print("Top", num+1, graph.nodes[L[i][0]])
        print("weight:", L[i][1])
        num += 1
def top_n_authority_for_nx(a, graph, n): 
    # a 為一個dictionary物件，有網站代碼跟其權重值 EX:(1294317, 0.0028239679462683694)
    # grpah 為 nx 套件的圖
    # n: 找前幾名 (int)
    L = sorted(a.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        print("Top", num+1, graph.nodes[L[i][0]])
        print("weight:", L[i][1])
        num += 1
def top_n_hub_for_nx(h, graph, n):
    L = sorted(h.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        if len(graph.nodes[L[i][0]]) >= 1:
            print("Top", num+1, graph.nodes[L[i][0]])
            print("weight:", L[i][1])
            num += 1

In [2]:
# 此城市適用於已經有圖以及其所有的關係！！！！！！
# 此程式用於將已建立之圖，去除其相同之網域，令其從相同網域指入之連結，"禁止連入"！

In [3]:
# 步驟一：要先讀出目標網域，規則：從https開始讀取到 第一個 "/" 為止。
# 步驟二：如果網域相同，那就將網域相同的網頁刪除(parent ,跟 children的網頁都要刪掉)
# 經過此處理，會將所有相同網域的網站都除去，只留下不同網域的連結。

In [4]:
# 測試正則化處理之後抓出的domain
csv.field_size_limit(500 * 1024 * 1024)
csv_file_name = "Graph0311_nineth_tier.csv" # 這邊放入原始圖
csvfile = open(csv_file_name, newline='', encoding = "utf-8")
# 讀取 CSV 檔案內容
rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
# 設定正則表達式模型
original_domain_md = re.compile(
    '(([a-zA-Z0-9\._-]+\.[a-zA-Z]{2,100})|([0-9]{1,100}|[\w]{2,100}\.[\d]{1,100}|[\d]{2,100}\.[\d]{1,100}|[\w]{2,100}\.[\d]{1,100}))'
)
# 以迴圈檢查檢查前11筆
#count = 0
all_node = []
for row in rows:
    node = []
    #if count == 11:
        #break
    #print('original',row[1]) # <class 'str'> # original url
    if row[1][:8] != "https://":
        continue # 跳過不儲存
    original_domain = original_domain_md.search(row[1])
    # if original_domain != None:
        # print(original_domain.group()) # <class "str">
    #檢查所有的 link
    # print(row[2]) # <class 'str'> # parent list
    parents_list = str_to_url(row[2])
    # print(type(parents_list))
    # 刪除domain相同的parent
    for index,parent in enumerate(parents_list):
        if parent[:8] != "https://":
            del parents_list[index]
        else:
            url_domain = original_domain_md.search(parent)
            try:
                if url_domain.group() == original_domain.group():
                    del parents_list[index]
            except Exception as e:
                pass
    # print(row[3]) # <class 'str'> # children list
    children_list = str_to_url(row[3])
    # 刪除domain相同的child
    for index2,child in enumerate(children_list):
        if child[:8] != "https://":
            del children_list[index2]
        else:    
            url_domain2 = original_domain_md.search(child)
            #print(url_domain.group())
            try:
                if url_domain2.group() == original_domain.group():
                    del children_list[index2]
            except Exception as e:
                pass
    #print(row[4]) # <class 'str'> # auth score
    #print(row[5]) # <class 'str'> # hub score
    #count += 1
    node.append(row[0])
    node.append(row[1])
    node.append(parents_list)
    node.append(children_list)
    if len(parents_list) != 0 or len (children_list) != 0: # 只要有一不為0就可以加入(雙為空擋下)
        all_node.append(node)
with open("Graph_self reference remove(0311).csv", 'w', newline='', encoding='UTF-8') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL,delimiter=';')
    writer.writerows(all_node)

In [5]:
# 檢查
print(all_node[:10])

[['74', 'https://', ['https://www.diabetesdaily.com', 'https://www.sartomer.com/zh/', 'https://www.snopes.com/about/', 'https://skepticalarrie.tumblr.com/post/679192734255448064/what-if-instead-of-the-door-opening-we-get-a-knock', 'https://equitablefutures.iftf.org/in-the-media/', 'https://139.59.195.188/slots/spadegaming', 'https://inmedase.org', 'https://www.ienearth.org/defenders-of-the-land-indigenous-peoples-have-clear-demands-for-real-change/', 'https://pages.nokia.com/T006V7-High-Accuracy-Indoor-Positioning-Digitalizing-assets-at-the-Chennai-Factory.html', 'https://getridhow.com', 'https://limburgcross.nl/', 'https://www.forane.com/zh/', 'https://www.snopes.com/contact/', 'https://www.gov.ie/en/service/be74d3-covid-19-pandemic-unemployment-payment', 'https://www.sartomer.com/ja/'], []], ['75', 'https:// http://sociologists.spb.ru', ['https://www.severreal.org/a/god-posle-yanvarskih-protestov-kak-izmenilos-obschestvo/31683836.html'], []], ['76', 'https:// https://careers-hackerno

In [6]:
# 開始實作

In [7]:
# 這邊需要建立 Sorted_All_list
All_list = []
csvfile = open("Graph_self reference remove(0311).csv", newline='', encoding = "utf-8")
rows = csv.reader(csvfile, delimiter=';')
for row in rows:
    if row[1][:4] == "http":
        All_list.append(row[1])
    for url in row[2]:
        if len(url) > 1:
            All_list.append(url)
    for url2 in row[3]:
        if len(url2) > 1:
            All_list.append(url2)
All_list = set(All_list)
Sorted_All_list = sorted(All_list)
print(len(Sorted_All_list))

1826665


In [8]:
len(Sorted_All_list[0])

8

In [9]:
HITS_r = nx.DiGraph() # 建立一個叫HITS的有向圖

In [10]:
count = 0
for index, url in enumerate(Sorted_All_list):
    if len(url) != 0:
        HITS_r.add_node(index, id = index ,original_url = url)
        count += 1
count

1826665

In [11]:
HITS_r._node

{0: {'id': 0, 'original_url': 'https://'},
 1: {'id': 1, 'original_url': 'https:// http://sociologists.spb.ru'},
 2: {'id': 2,
  'original_url': 'https:// https://careers-hackernoon.paperform.co/'},
 3: {'id': 3, 'original_url': 'https:// www.adition.com/kontakt/impressum/'},
 4: {'id': 4, 'original_url': 'https:// www.tevi.co.uk'},
 5: {'id': 5, 'original_url': 'https://#'},
 6: {'id': 6, 'original_url': 'https://#####.instagram.com/'},
 7: {'id': 7, 'original_url': 'https://#####.pinterest.com/'},
 8: {'id': 8, 'original_url': 'https://#####twitter.com/'},
 9: {'id': 9, 'original_url': 'https://####.facebook.com/'},
 10: {'id': 10, 'original_url': 'https://####.linkedin.com/'},
 11: {'id': 11,
  'original_url': 'https://${settings.INSTITUTE_HOST}/Copyright'},
 12: {'id': 12,
  'original_url': 'https://%D1%80%D0%BE%D1%81%D1%81%D0%B8%D1%8F%D1%81%D0%B5%D0%B3%D0%BE%D0%B4%D0%BD%D1%8F.%D1%80%D1%84/#productsandservices'},
 13: {'id': 13,
  'original_url': 'https://%D1%80%D0%BE%D1%81%D1%81%D

In [12]:
help(HITS_r)

Help on DiGraph in module networkx.classes.digraph object:

class DiGraph(networkx.classes.graph.Graph)
 |  DiGraph(incoming_graph_data=None, **attr)
 |  
 |  Base class for directed graphs.
 |  
 |  A DiGraph stores nodes and edges with optional data, or attributes.
 |  
 |  DiGraphs hold directed edges.  Self loops are allowed but multiple
 |  (parallel) edges are not.
 |  
 |  Nodes can be arbitrary (hashable) Python objects with optional
 |  key/value attributes. By convention `None` is not used as a node.
 |  
 |  Edges are represented as links between nodes with optional
 |  key/value attributes.
 |  
 |  Parameters
 |  ----------
 |  incoming_graph_data : input graph (optional, default: None)
 |      Data to initialize graph. If None (default) an empty
 |      graph is created.  The data can be any format that is supported
 |      by the to_networkx_graph() function, currently including edge list,
 |      dict of dicts, dict of lists, NetworkX graph, NumPy matrix
 |      or 2d n

In [13]:
start_time = time.time()
# 開始建立關係
csv.field_size_limit(500 * 1024 * 1024)
with open("Graph_self reference remove(0311).csv", newline='', encoding = "utf-8") as csvfile: # 開啟之前做的圖檔
    # 讀取 CSV 檔案內容
    #count = 0
    rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
    for row in rows:
        #if count == 100000:
        #    break
        parents_list = str_to_url(row[2])
        if len(parents_list) != 0: # in_link 不為0才做
            for in_link in parents_list:
                index = binary_search_for_nx(Sorted_All_list, in_link)
                if index != "Node Not Found!":
                    original_id = int(row[0])
                    HITS_r.add_edge(index, original_id)
        children_list = str_to_url(row[3])
        if len(children_list) != 0: #out_link集合不為0才做
            for out_link in children_list:
                index2 = binary_search_for_nx(Sorted_All_list, out_link)
                # print(binary_search_for_nx(all_original_list, row[1]))
                if index2 != "Node Not Found!":
                    #print(row[0]) #row[0] <class: str>
                    original_id = int(row[0])
                    HITS_r.add_edge(original_id, index2)  # type int, type int
                    # 上面一行加入的點為 原始的id 跟 index
                    # 注意，這一行如果新增的點不再圖中，會自動新增   
        #count += 1
print("花費:" + str(float(time.time() - start_time)) + "秒")

花費:157.9315767288208秒


In [14]:
len(HITS_r.edges)

8288976

In [15]:
binary_search_for_nx(Sorted_All_list, "https://dictionary.cambridge.org/zht/%E8%A9%9E%E5%85%B8/%E8%8B%B1%E8%AA%9E/great")

243031

In [16]:
len(HITS_r.nodes)

2540641

# HITS

In [63]:
start_time = time.time()
h,a=nx.hits(HITS_r, max_iter = 300 , tol = 0.000000000000000000001 , normalized = True) # 得到兩個dictionary(未排序)
print("花費:" + str(float(time.time() - start_time)) + "秒")

花費:64.1171007156372秒


In [64]:
help(nx.hits)

Help on function hits in module networkx.algorithms.link_analysis.hits_alg:

hits(G, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    Returns HITS hubs and authorities values for nodes.
    
    The HITS algorithm computes two numbers for a node.
    Authorities estimates the node value based on the incoming links.
    Hubs estimates the node value based on outgoing links.
    
    Parameters
    ----------
    G : graph
      A NetworkX graph
    
    max_iter : integer, optional
      Maximum number of iterations in power method.
    
    tol : float, optional
      Error tolerance used to check convergence in power method iteration.
    
    nstart : dictionary, optional
      Starting value of each node for power method iteration.
    
    normalized : bool (default=True)
       Normalize results by the sum of all of the values.
    
    Returns
    -------
    (hubs,authorities) : two-tuple of dictionaries
       Two dictionaries keyed by node containing the hub and auth

In [65]:
HITS_r.nodes[129]

{'id': 129, 'original_url': 'https://010-onderwijs.startpagina.nl/'}

In [66]:
top_n_authority_for_nx(a, HITS_r, 10000)

Top 1 {'id': 1377195, 'original_url': 'https://www.flickr.com/photos/worldbank'}
weight: 0.0033892118736906806
Top 2 {'id': 687102, 'original_url': 'https://olc.worldbank.org'}
weight: 0.0033853404439773962
Top 3 {'id': 174909, 'original_url': 'https://clientconnection.worldbank.org/servlet/main?menuPK=234032&pagePK=232663&piPK=234033'}
weight: 0.003384373582451547
Top 4 {'id': 687103, 'original_url': 'https://olc.worldbank.org/'}
weight: 0.0033824040359195216
Top 5 {'id': 737507, 'original_url': 'https://policies.worldbank.org/sites/PPF3/Pages/PPFHome.aspx'}
weight: 0.0033504533524517954
Top 6 {'id': 1778548, 'original_url': 'https://www.worldbank.org/en/projects-operations/products-and-services/procurement-projects-programs'}
weight: 0.003335346477969559
Top 7 {'id': 1001826, 'original_url': 'https://treasury.worldbank.org/'}
weight: 0.00329926094845376
Top 8 {'id': 1777965, 'original_url': 'https://www.worldbank.org/en/about/unit/integrity-vice-presidency'}
weight: 0.003271645640380

weight: 5.6669050461813285e-05
Top 932 {'id': 1764175, 'original_url': 'https://www.vsemirnyjbank.org/ru/about/legal'}
weight: 5.6520175562567225e-05
Top 933 {'id': 1189021, 'original_url': 'https://www.albankaldawli.org/ar/about/people/all'}
weight: 5.633780388633446e-05
Top 934 {'id': 1635346, 'original_url': 'https://www.shihang.org/zh/news'}
weight: 5.6166935923293283e-05
Top 935 {'id': 1635338, 'original_url': 'https://www.shihang.org/zh/access-to-information'}
weight: 5.593776457935887e-05
Top 936 {'id': 1764237, 'original_url': 'https://www.vsemirnyjbank.org/ru/results'}
weight: 5.5724744135520615e-05
Top 937 {'id': 1189031, 'original_url': 'https://www.albankaldawli.org/ar/country/gcc'}
weight: 5.56131213043306e-05
Top 938 {'id': 1741646, 'original_url': 'https://www.twitter.com/BM_Afrique'}
weight: 5.496368855336009e-05
Top 939 {'id': 1212177, 'original_url': 'https://www.bancomundial.org/es/who-we-are/site-accessibility'}
weight: 5.482321060167473e-05
Top 940 {'id': 1777890, 

weight: 1.2104210492467904e-05
Top 1832 {'id': 863204, 'original_url': 'https://search.worldbank.org/api/v2/news?format=atom&countrycode_exact=CR'}
weight: 1.2103047908957543e-05
Top 1833 {'id': 255995, 'original_url': 'https://documentos.bancomundial.org/es/publication/documents-reports/documentlist?query=&count_key=82630&lang_key=120706&majdocty_key=906674^658101'}
weight: 1.2103047867708408e-05
Top 1834 {'id': 221029, 'original_url': 'https://datos.bancomundial.org/pais/costa-rica'}
weight: 1.2102894463216306e-05
Top 1835 {'id': 255940, 'original_url': 'https://documentos.bancomundial.org/curated/es/2015/05/24447368/costa-rica-systematic-country-diagnostic'}
weight: 1.2102894463216306e-05
Top 1836 {'id': 755792, 'original_url': 'https://projects.bancomundial.org/es/projects-operations/project-detail/P123146'}
weight: 1.2102894463216306e-05
Top 1837 {'id': 755806, 'original_url': 'https://projects.bancomundial.org/search?lang=es&searchTerm=&countrycode_exact=CR'}
weight: 1.2102894463

Top 2731 {'id': 219524, 'original_url': 'https://data.albankaldawli.org/country/YE'}
weight: 6.2654291977232705e-06
Top 2732 {'id': 256403, 'original_url': 'https://documents.worldbank.org/en/publication/documents-reports/documentdetail/594461562050901672/republic-of-yemen-country-engagement-note-for-the-period-fy20-fy21'}
weight: 6.2654291977232705e-06
Top 2733 {'id': 755728, 'original_url': 'https://projects.albankaldawli.org/ar/projects-operations/project-detail/P173862'}
weight: 6.2654291977232705e-06
Top 2734 {'id': 755737, 'original_url': 'https://projects.albankaldawli.org/search?lang=ar&searchTerm=&countrycode_exact=RY'}
weight: 6.2654291977232705e-06
Top 2735 {'id': 117541, 'original_url': 'https://blogs.worldbank.org/search?f%5B0%5D=countries%3A141&f%5B1%5D=language%3Aar'}
weight: 6.265343518624013e-06
Top 2736 {'id': 1143957, 'original_url': 'https://web.worldbank.org/WBSITE/EXTERNAL/DEVCOMMEXT/0pagePK:64000837~piPK:64001152~theSitePK:277473~contentMDK:2299249800.html'}
weig

Top 3684 {'id': 1429511, 'original_url': 'https://www.ifc.org/wps/wcm/connect/corp_ext_content/ifc_external_corporate_site/about+ifc_new/leadership/hela-cheikhrouhou'}
weight: 5.727897544232323e-06
Top 3685 {'id': 256550, 'original_url': 'https://documents1.worldbank.org/curated/en/116841606928570369/pdf/Realizing-the-Blue-Economy-Potential-in-Albania.pdf'}
weight: 5.727865296606193e-06
Top 3686 {'id': 256558, 'original_url': 'https://documents1.worldbank.org/curated/en/324981524464014452/pdf/124573-ALBANIAN-PUBLIC-PEFA-AL-web.pdf'}
weight: 5.727865296606193e-06
Top 3687 {'id': 1777935, 'original_url': 'https://www.worldbank.org/en/about/people/cyril-muller'}
weight: 5.727853728475843e-06
Top 3688 {'id': 1341813, 'original_url': 'https://www.facebook.com/WorldBankNepal'}
weight: 5.727546865819757e-06
Top 3689 {'id': 1143945, 'original_url': 'https://web.worldbank.org/WBSITE/EXTERNAL/BANCOMUNDIAL/NEWSSPANISH/0contentMDK:21458441~pagePK:64257043~piPK:437376~theSitePK:107456800.html'}
wei

...

Top 5631 {'id': 1778256, 'original_url': 'https://www.worldbank.org/en/news/all?countrycode_exact=UZ&apilang=ru&lang_exact=Russian&isresult=true'}
weight: 1.311347268631902e-06
Top 5632 {'id': 117532, 'original_url': 'https://blogs.worldbank.org/ru/europeandcentralasia/uzbekistan-timely-response-learning-during-school-closures'}
weight: 1.3113440778490198e-06
Top 5633 {'id': 117561, 'original_url': 'https://blogs.worldbank.org/search?f%5B0%5D=countries%3A264&f%5B1%5D=language%3Aru'}
weight: 1.3113327427075563e-06
Top 5634 {'id': 755974, 'original_url': 'https://projects.vsemirnyjbank.org/search?lang=ru&searchTerm=&countrycode_exact=UZ'}
weight: 1.3113327259542663e-06
Top 5635 {'id': 220295, 'original_url': 'https://data.worldbank.org/country/uzbekistan/russian'}
weight: 1.3113161013297615e-06
Top 5636 {'id': 256208, 'original_url': 'https://documents.vsemirnyjbank.org/ru/publication/documents-reports/documentdetail/516181470070366635'}
weight: 1.3113161013297615e-06
Top 5637 {'id': 256

Top 6631 {}
weight: 9.62102489700492e-07
Top 6632 {'id': 1402168, 'original_url': 'https://www.gov.tw/News3.aspx?n=2&sms=9037'}
weight: 9.621024057130503e-07
Top 6633 {'id': 1241300, 'original_url': 'https://www.books.com.tw/products/N011797294?loc=P_E211230000000214_title_009'}
weight: 9.621015013453058e-07
Top 6634 {'id': 1674693, 'original_url': 'https://www.techtarget.com/searchstorage/definition/JEDEC'}
weight: 9.620934222083025e-07
Top 6635 {'id': 915224, 'original_url': 'https://spikeymagpie.tumblr.com/post/679183474117181440/notajerusalemcricket-anomalous-heretic'}
weight: 9.620933429032946e-07
Top 6636 {'id': 128215, 'original_url': 'https://boychic.tumblr.com/post/679167626255220736'}
weight: 9.620925320722907e-07
Top 6637 {}
weight: 9.620919727371202e-07
Top 6638 {'id': 135915, 'original_url': 'https://buildd.debian.org/status/package.php?p=libposix-2008-perl'}
weight: 9.620916019535054e-07
Top 6639 {'id': 650474, 'original_url': 'https://napidoktor.hu/allergia/allergias-rhi

Top 7531 {'id': 896685, 'original_url': 'https://sites.google.com/view/jejudom0624j/'}
weight: 9.612958050547028e-07
Top 7532 {'id': 1072473, 'original_url': 'https://twitter.com/intent/tweet?text=bokettochild%3A%20redrikki%3A%20asexual-musings%3A%20asexual-musings%3A%20no%20one%3A%20asexuals%20before%20realizing%20they%E2%80%99re%20ace%3A%20isn%E2%80%99t%20it%20weird%20how%20we%20call%20people%20%E2%80%9Chot%E2%80%9D%20when%20we%20just%20mean%2C%20like%2C%20pretty%3F%20it%20makes%20it%20sound%20like%20you%20actually%20feel%20hot%20when%20you%20see%20them....%20https%3A%2F%2Ftmblr.co%2FZwQoFdbi_SE-Sq00'}
weight: 9.61295783638908e-07
Top 7533 {'id': 170451, 'original_url': 'https://cifjobs.ie/advice/'}
weight: 9.61295686068249e-07
Top 7534 {'id': 880487, 'original_url': 'https://shared-drupal-s3fs.s3-ap-southeast-2.amazonaws.com/master-test/fapub_pdf/00+-+Planning+Portal+Exhibitions/Major+Projects+reform/Industry+Specific+SEARs+-+Hospitals.pdf'}
weight: 9.612950941465928e-07
Top 7535 {'

weight: 9.605999592496262e-07
Top 8531 {'id': 1686519, 'original_url': 'https://www.tn-sanso.co.jp/en/'}
weight: 9.605996292552369e-07
Top 8532 {'id': 1686521, 'original_url': 'https://www.tn-sanso.co.jp/en/csr/soccerclinic.html'}
weight: 9.605996292552369e-07
Top 8533 {'id': 1396846, 'original_url': 'https://www.google.com.br/maps/place/PROF+FRANCISCO+FONSECA++379+Saquarema+-+RJ'}
weight: 9.605992784071182e-07
Top 8534 {'id': 1396882, 'original_url': 'https://www.google.com.br/maps/place/R+CAP+FRANCISCO+CABRAL+109++109+Mendes+-+RJ'}
weight: 9.605992784071182e-07
Top 8535 {'id': 1396896, 'original_url': 'https://www.google.com.br/maps/place/R+DEP+FRANCELINO+BASTOS+FRANCA+137+Varre-sai+-+RJ'}
weight: 9.605992784071182e-07
Top 8536 {'id': 1396901, 'original_url': 'https://www.google.com.br/maps/place/R+DO+COMERCIO+LOJA+B+233+Angra+dos+Reis+-+RJ'}
weight: 9.605992784071182e-07
Top 8537 {'id': 254290, 'original_url': 'https://docs.microsoft.com/en-us/windows/win32/winauto/entry-uiauto-win3

weight: 9.6055572934605e-07
Top 9531 {'id': 915822, 'original_url': 'https://spoco.tumblr.com/post/678784264532328448/bananatwinky-when-i-get-rejected-by-another-job'}
weight: 9.60555726905123e-07
Top 9532 {'id': 915823, 'original_url': 'https://spoco.tumblr.com/post/678814629060853760'}
weight: 9.60555726905123e-07
Top 9533 {'id': 1535304, 'original_url': 'https://www.mybloggerclub.com/category/entertainment/movie/'}
weight: 9.605557257830982e-07
Top 9534 {'id': 1465644, 'original_url': 'https://www.jobvite.com/solutions/use-case/onboarding/'}
weight: 9.605557254935548e-07
Top 9535 {'id': 1160187, 'original_url': 'https://workforce.libretexts.org/'}
weight: 9.605557254879964e-07
Top 9536 {'id': 849066, 'original_url': 'https://sapphicIFF.com'}
weight: 9.605557248375058e-07
Top 9537 {'id': 1560903, 'original_url': 'https://www.niesr.ac.uk/publication-type/global-economic-outlook'}
weight: 9.60555720694421e-07
Top 9538 {'id': 946146, 'original_url': 'https://sustainabledevelopment.un.or

In [67]:
# 0314
# 嘗試把網域中有org的部分存出來
def remove_particular_domain_auth(a, graph, n):
    L = sorted(a.items(), key = lambda item:item[1], reverse=True)
    num = 0
    domain_md = re.compile(
        '(([a-zA-Z0-9\._-]+\.[a-zA-Z]{2,100})|([0-9]{1,100}|[\w]{2,100}\.[\d]{1,100}|[\d]{2,100}\.[\d]{1,100}|[\w]{2,100}\.[\d]{1,100}))'
    )
    # 計算為.org結尾的網址數目(.org結尾大部分為world bank)
    count_org = 0
    for i in range(n):
        try:
            url = graph.nodes[L[i][0]]["original_url"]
            original_domain = domain_md.search(url)
            if original_domain.group()[-4:] == ".org":
                count_org += 1
            else:
                print("top", i+1, url)
        except:
            continue
        #url_domain = domain_md.search(graph.nodes[L[i][0]])
        #if len(url_domain.group()) > 4:
        #print(url_domain.group())
        num += 1
    print(".org appear" ,count_org ,"times")

In [68]:
remove_particular_domain_auth(a, HITS_r,10000)

top 1 https://www.flickr.com/photos/worldbank
top 42 https://www.linkedin.com/company/the-world-bank
top 56 https://www.youtube.com/user/WorldBank
top 113 https://instagram.com/worldbank/
top 120 https://www.facebook.com/worldbank
top 121 https://www.twitter.com/worldbank
top 422 https://wbgcmsprod.microsoftcrmportals.com/en-US/anonymous-users/int-fraud-management/create-new-complaint/
top 514 https://instagram.com/banquemondiale/
top 540 https://www.twitter.com/banquemondiale
top 541 https://www.facebook.com/banquemondiale
top 545 https://wbgcmsprod.microsoftcrmportals.com/fr-FR/anonymous-users/int-fraud-management/create-new-complaint/
top 564 https://www.youtube.com/playlist?list=PL7C2995270746C48F
top 565 https://instagram.com/bancomundial/
top 588 https://twitter.com/Albankaldawli
top 589 https://www.facebook.com/AlbankAldawli
top 590 https://www.instagram.com/albankaldawli
top 591 https://www.youtube.com/playlist?list=PL5E7BABB475E47699
top 592 https://wbgcmsprod.microsoftcrmport

top 6117 https://cbos.tas.gov.au/topics/licensing-and-registration/licensed-occupations/owner-builder-permit/restrictions
top 6118 https://www.ravemobilesafety.com/products/rave-guardian/
top 6120 https://graphisoft.com/us/press-release-archicad-25-update-2-announced
top 6122 https://wiwin.de/produkt/inyova
top 6123 https://reddit.com/submit?url=https%3A%2F%2Faporupasworld.tumblr.com%2Fpost%2F666487811810164736%3Fis_related_post%3D1
top 6124 https://news.cornell.edu/stories/2021/03/three-emerging-scholars-lead-new-research-partnerships
top 6125 https://business.financialpost.com/category/news/
top 6126 https://reddit.com/submit?url=https%3A%2F%2Frndmpersons.tumblr.com%2Fpost%2F679213638755794944%2Frhyperographer-glumshoe-glumshoe-this-was-my
top 6127 https://r.search.yahoo.com/cbclk2/dWU9MEU2ODhBQjJFMjAyNDI1NCZ1dD0xNjQzODc1NDQyMTM1JnVvPTc2MTQxMzM1OTQyNDE5Jmx0PTImcz0xJmVzPU9GLmEuejRHUFM5bVJJUmhDZDRJbERWWTNkTlRBV2p3NjJrOVFLNWgyQnZoZHVRRw--/RV=2/RE=1643904242/RO=10/RU=https%3a%2f%2fwww.bi

top 7807 https://phileo-lesaffre.com/en/beef/beef-weight-gain-feed/
top 7808 https://onzetaal.nl/tijdschrift/inhoudsopgaven/
top 7809 https://twitter.com/intent/tweet?url=&text=Does Overstock take Klarna for financing? — Knoji&via=askknoji
top 7810 https://radiopublic.com/martech-zone-interviews-Wa3zDb
top 7811 https://camunda.com/
top 7815 https://fiuonline.fiu.edu/
top 7819 https://datscha.co.uk/2020/10/07/press-release-datscha-expands-product-suite/
top 7825 https://celepeace.tumblr.com/
top 7826 https://uselessbogwitch.tumblr.com/post/678720904060141568
top 7828 https://fuhumedia.blogspot.com/search/label/Heller%20%C3%81gnes
top 7829 https://fuhumedia.blogspot.com/search/label/Helsingborg
top 7830 https://fuhumedia.blogspot.com/search/label/AFP
top 7831 https://appletvplus.knoji.com/questions/appletvplus-order-tracking/
top 7832 https://atttvnow.knoji.com/questions/atttvnow-bread-financing1/
top 7833 https://churchstreamingtv.knoji.com/questions/churchstreamingtv-4k-content/
top 78

top 9474 https://www.nutraceuticalsworld.com/contents/view_breaking-news/2022-03-18/clinical-study-finds-plant-based-omega-3-ingredient-efficacious/
top 9475 https://olark.com/ada-accessible-live-chat
top 9476 https://olark.com/pricing
top 9477 https://chancefxrzj.therainblog.com/11405228/examine-this-report-on-black-magic-love-spells-that-work-immediately
top 9478 https://www.highsnobiety.com/p/cafe-du-cycliste-66onorth-collection/
top 9480 https://devinxvqkd.therainblog.com/11405146/reasonably-priced-website-positioning-rankings
top 9481 https://ophanim-vesper.tumblr.com/post/670270563014656000
top 9482 https://graphisoft.com/hu/resources-and-support/system-requirements
top 9483 https://tr.betboo253.com/t/info/social-responsibility/responsible-gaming.aspx
top 9484 https://tr.betboo253.com/t/info/social-responsibility/underage-gambling.aspx
top 9485 https://shop.popularmechanics.com/menshealth-magazine.html
top 9486 https://businesswales.gov.wales/skillsgateway/skills-and-training-pro

In [69]:
top_n_hub_for_nx(h, HITS_r, 10000)

Top 1 {'id': 445660, 'original_url': 'https://help.yahoo.com/kb/account/reset-yahoo-password-sln27051.html'}
weight: 0.0023541903520565037
Top 2 {'id': 445667, 'original_url': 'https://help.yahoo.com/kb/finance-for-web/SLN2310.html?locale=en_US'}
weight: 0.002353856289438765
Top 3 {'id': 445755, 'original_url': 'https://helpbristolshomeless.org/hbh-job-opportunities/'}
weight: 0.0023538114345627126
Top 4 {'id': 445734, 'original_url': 'https://help.zenodo.org'}
weight: 0.0023538114304441885
Top 5 {'id': 445695, 'original_url': 'https://help.yahoo.com/kb/index?page=product&locale=vi_VN&y=PROD_ACCT'}
weight: 0.002353811430331567
Top 6 {'id': 445692, 'original_url': 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US'}
weight: 0.002353811430063098
Top 7 {'id': 445694, 'original_url': 'https://help.yahoo.com/kb/index?page=product&locale=fr_CA&y=PROD_SRCH'}
weight: 0.002353811430063098
Top 8 {'id': 445676, 'original_url': 'https://help.yahoo.com/kb/index?locale=en_PH&

Top 800 {'id': 194390, 'original_url': 'https://copper-claw.tumblr.com/'}
weight: 4.853934919971045e-06
Top 801 {'id': 719791, 'original_url': 'https://pillowgraveyard.tumblr.com/post/661514608591388672/oyasumi-punpun-%E3%81%8A%E3%82%84%E3%81%99%E3%81%BF%E3%83%97%E3%83%B3%E3%83%97%E3%83%B3'}
weight: 4.8301180766159705e-06
Top 802 {'id': 1321185, 'original_url': 'https://www.envdesigngroup.com/community-impact-across-the-pond/#comment-50648'}
weight: 4.808544773283478e-06
Top 803 {'id': 361400, 'original_url': 'https://ferrjs-wheel.tumblr.com/'}
weight: 4.808370586916505e-06
Top 804 {'id': 602718, 'original_url': 'https://mcalistersdeli.knoji.com/questions/mcalistersdeli-order-changescancellations/'}
weight: 4.725671817607373e-06
Top 805 {'id': 602720, 'original_url': 'https://mcalistersdeli.knoji.com/questions/mcalistersdeli-positive-environmental-practices/'}
weight: 4.725671817607373e-06
Top 806 {'id': 602721, 'original_url': 'https://mcalistersdeli.knoji.com/questions/mcalistersdeli

Top 1514 {'id': 1552141, 'original_url': 'https://www.ncbi.nlm.nih.gov/portal/signin.fcgi?cmd=NcbilsSignin&auth_id=university_of_applied_sciences_for_health_professions_upper_austria&rrme=0&surl=https%3A%2F%2Fwww.ncbi.nlm.nih.gov%2Faccount%2Flogincheck%2F%3Fback_url%3Dhttps%253A%252F%252Fpubmed.ncbi.nlm.nih.gov%252F27929415%252F%2523add-to-favorites&nurl=https%3A%2F%2Fwww.ncbi.nlm.nih.gov%2Faccount%2FlinkAccount%2F%3Fback_url%3Dhttps%253A%252F%252Fpubmed.ncbi.nlm.nih.gov%252F27929415%252F%2523add-to-favorites&furl=https%3A%2F%2Fwww.ncbi.nlm.nih.gov%2Faccount%2F%3Fcmd%3DsigninError'}
weight: 2.371233147257087e-07
Top 1515 {'id': 1551906, 'original_url': 'https://www.ncbi.nlm.nih.gov/portal/signin.fcgi?cmd=NcbilsSignin&auth_id=universite_de_bretagne_occidentale_-_ubo_idp3__france&rrme=0&surl=https%3A%2F%2Fwww.ncbi.nlm.nih.gov%2Faccount%2Flogincheck%2F%3Fback_url%3Dhttps%253A%252F%252Fwww.ncbi.nlm.nih.gov%252Fmyncbi%252F&nurl=https%3A%2F%2Fwww.ncbi.nlm.nih.gov%2Faccount%2FlinkAccount%2F%3

Top 2500 {'id': 239190, 'original_url': 'https://developers.google.com/assistant/app/overview'}
weight: 1.1837923709107426e-07
Top 2501 {'id': 1453074, 'original_url': 'https://www.instagram.com/zerowasteucla/'}
weight: 1.1837667536715837e-07
Top 2502 {'id': 278694, 'original_url': 'https://dysonshop.blob.core.windows.net/products/dyson-v12-detect-slim™-total-clean/gallery/1-100x60.jpg'}
weight: 1.1837469935140007e-07
Top 2503 {'id': 1245365, 'original_url': 'https://www.bramkasse.nl'}
weight: 1.183729135478191e-07
Top 2504 {'id': 295222, 'original_url': 'https://emtemp.gcom.cloud/ngw/globalassets/en/about/documents/gartner-csr-report-2019.pdf'}
weight: 1.183691363787714e-07
Top 2505 {'id': 1087614, 'original_url': 'https://twitter.com/intent/tweet?text=wikipediagf%3A%20I%E2%80%99m%20just%20like%20tv%20static%20but%20a%20girl%20https%3A%2F%2Ftmblr.co%2FZxRJHUbi-2590e00'}
weight: 1.1836661175158367e-07
Top 2506 {'id': 1215192, 'original_url': 'https://www.bblat.se/info/om-bbl-at-och-bon

Top 3400 {'id': 600578, 'original_url': 'https://mateinternetandmobile.knoji.com/questions/mateinternetandmobile-amazon-pay/'}
weight: 4.9215803691801966e-08
Top 3401 {'id': 940536, 'original_url': 'https://superslot1234-5070370.theblogfairy.com/10314077/considerations-to-know-about-1234-superslot-%E0%B9%80%E0%B8%84%E0%B8%A3%E0%B8%94-%E0%B8%95%E0%B8%9F%E0%B8%A3-50'}
weight: 4.921271247801621e-08
Top 3402 {'id': 177867, 'original_url': 'https://clusterduck28.tumblr.com/post/679043138890924032'}
weight: 4.9155975716773424e-08
Top 3403 {'id': 1384006, 'original_url': 'https://www.fsb-tcfd.org/'}
weight: 4.9136794405344155e-08
Top 3404 {'id': 973792, 'original_url': 'https://thedamnephilimfangirl.tumblr.com/post/679122987238244352'}
weight: 4.9059434969312254e-08
Top 3405 {'id': 1429149, 'original_url': 'https://www.iep.utm.edu/noz-poli/#SH3i'}
weight: 4.902710132610717e-08
Top 3406 {'id': 497865, 'original_url': 'https://interiordefine.knoji.com/questions/interiordefine-shop-pay/'}
weight

Top 4300 {'id': 499338, 'original_url': 'https://intheheartofman.tumblr.com/'}
weight: 4.254397340831625e-08
Top 4301 {'id': 999804, 'original_url': 'https://transformativeworks.tumblr.com'}
weight: 4.254160512727978e-08
Top 4302 {'id': 135844, 'original_url': 'https://buildd.debian.org/status/package.php?p=libmouse-perl'}
weight: 4.254145105793909e-08
Top 4303 {'id': 135864, 'original_url': 'https://buildd.debian.org/status/package.php?p=libnet-libdnet-perl'}
weight: 4.254145105793909e-08
Top 4304 {'id': 150963, 'original_url': 'https://careers.umgc.edu/'}
weight: 4.254145105793909e-08
Top 4305 {'id': 151073, 'original_url': 'https://careers.weill.cornell.edu'}
weight: 4.254145105793909e-08
Top 4306 {'id': 151264, 'original_url': 'https://cargasenergy.com'}
weight: 4.254145105793909e-08
Top 4307 {'id': 151324, 'original_url': 'https://caring-is-creeepy-blog.tumblr.com/'}
weight: 4.254145105793909e-08
Top 4308 {'id': 151602, 'original_url': 'https://carnegiesciencecenter.knoji.com/ques

Top 5282 {'id': 1037741, 'original_url': 'https://twitter.com/NealShusterman/'}
weight: 3.2708911941762366e-08
Top 5283 {'id': 485279, 'original_url': 'https://iminthebandironweaselband.fandom.com/wiki/'}
weight: 3.270545839195141e-08
Top 5284 {'id': 1825631, 'original_url': 'https://zoneoftheenders.fandom.com/wiki/'}
weight: 3.269908766678745e-08
Top 5285 {'id': 32493, 'original_url': 'https://agt.fandom.com/wiki/'}
weight: 3.268980870300507e-08
Top 5286 {'id': 744833, 'original_url': 'https://prettylittleliars.fandom.com/wiki/'}
weight: 3.26885532314289e-08
Top 5287 {'id': 80749, 'original_url': 'https://avpm.fandom.com/wiki/'}
weight: 3.2680650939723644e-08
Top 5288 {'id': 834761, 'original_url': 'https://roadies.fandom.com/wiki/'}
weight: 3.267468572232999e-08
Top 5289 {'id': 413047, 'original_url': 'https://globalrose.knoji.com/questions/globalrose-flower-care-instructions/'}
weight: 3.267439258479914e-08
Top 5290 {'id': 273954, 'original_url': 'https://duck-dynasty.fandom.com/wik

weight: 2.4561910266376856e-08
Top 6200 {'id': 255078, 'original_url': 'https://docs.rs/tempdir/'}
weight: 2.4553571693782844e-08
Top 6201 {'id': 1430071, 'original_url': 'https://www.ign.com/games/kengo-3'}
weight: 2.455330941557824e-08
Top 6202 {'id': 658730, 'original_url': 'https://news.cornell.edu/stories/2021/10/postdoctoral-fellow-works-save-endangered-hawaiian-birds-mosquito-menace'}
weight: 2.4531327710592332e-08
Top 6203 {'id': 603305, 'original_url': 'https://mdpiblog.wordpress.sciforum.net/archives'}
weight: 2.452983958576401e-08
Top 6204 {'id': 605538, 'original_url': 'https://medikus.lap.hu/'}
weight: 2.4475991077549407e-08
Top 6205 {'id': 919145, 'original_url': 'https://sseoulful.tumblr.com/'}
weight: 2.44736756500714e-08
Top 6206 {'id': 1821968, 'original_url': 'https://zapsoda.tumblr.com/'}
weight: 2.447148176060361e-08
Top 6207 {'id': 1821969, 'original_url': 'https://zapsoda.tumblr.com/post/679164710516588544'}
weight: 2.447148176060361e-08
Top 6208 {'id': 1821971, 

Top 7071 {'id': 738226, 'original_url': 'https://polpsyispp.medium.com/?source=post_internal_links---------6-------------------------------'}
weight: 2.2970032452429596e-08
Top 7072 {'id': 738227, 'original_url': 'https://polpsyispp.medium.com/a-failed-welcome-6bdff91af7dc?source=post_internal_links---------6-------------------------------'}
weight: 2.2970032452429596e-08
Top 7073 {'id': 738893, 'original_url': 'https://ponte.org/durante-pandemia-policia-do-rio-diminui-operacoes-e-mortes-em-favelas-caem-60/'}
weight: 2.2970032452429596e-08
Top 7074 {'id': 738894, 'original_url': 'https://ponte.org/rj-registra-mais-de-6-operacoes-policiais-com-pelo-menos-uma-morte-por-dia-no-governo-witzel/'}
weight: 2.2970032452429596e-08
Top 7075 {'id': 738979, 'original_url': 'https://pood.aripaev.ee/koolitus-power-bi-dax-keel-paevane?utm_medium=koolitused&utm_source=aripaev.ee'}
weight: 2.2970032452429596e-08
Top 7076 {'id': 741377, 'original_url': 'https://positivepsychology.com/goal-setting-psycho

weight: 2.1393430249429024e-08
Top 7900 {'id': 670891, 'original_url': 'https://nl.ign.com/'}
weight: 2.1380647543675027e-08
Top 7901 {'id': 1252445, 'original_url': 'https://www.bworldonline.com/smooth-transition-may-help-barmm-region-attract-more-investments/'}
weight: 2.1380188104009638e-08
Top 7902 {'id': 310591, 'original_url': 'https://esd.ny.gov/certified-business-incubator'}
weight: 2.1377807777434615e-08
Top 7903 {'id': 163003, 'original_url': 'https://chancegatkb.theblogfairy.com/10317020/the-smart-trick-of-sports-betting-site-that-no-one-is-discussing'}
weight: 2.1377298980602287e-08
Top 7904 {'id': 948071, 'original_url': 'https://swiftsnowmane.tumblr.com/'}
weight: 2.1375320966241857e-08
Top 7905 {'id': 360014, 'original_url': 'https://felicitasz.blog.hu/2011/06/13/egy_orvos_levele_egy_rakkal_frissen_diagnosztizalt_paciensnek'}
weight: 2.136738527068095e-08
Top 7906 {'id': 1583408, 'original_url': 'https://www.peak360it.com/'}
weight: 2.1365287589998518e-08
Top 7907 {'id':

# PageRank

In [70]:
help(nx.pagerank)

Help on function pagerank in module networkx.algorithms.link_analysis.pagerank_alg:

pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1e-06, nstart=None, weight='weight', dangling=None)
    Returns the PageRank of the nodes in the graph.
    
    PageRank computes a ranking of the nodes in the graph G based on
    the structure of the incoming links. It was originally designed as
    an algorithm to rank web pages.
    
    Parameters
    ----------
    G : graph
      A NetworkX graph.  Undirected graphs will be converted to a directed
      graph with two directed edges for each undirected edge.
    
    alpha : float, optional
      Damping parameter for PageRank, default=0.85.
    
    personalization: dict, optional
      The "personalization vector" consisting of a dictionary with a
      key some subset of graph nodes and personalization value each of those.
      At least one personalization value must be non-zero.
      If not specfiied, a nodes personalization 

In [71]:
pagerank = nx.pagerank(HITS_r, alpha= 0.9)

In [72]:
top_n_pagerank_for_nx(pagerank, HITS_r, 100)

Top 1 {'id': 1273608, 'original_url': 'https://www.cloudflare.com/5xx-error-landing'}
weight: 0.0003588120902928459
Top 2 {}
weight: 0.0002202467887061409
Top 3 {'id': 625731, 'original_url': 'https://milfgender.tumblr.com/post/678742898455592960'}
weight: 5.27259572866183e-05
Top 4 {'id': 1518547, 'original_url': 'https://www.mediawiki.org/'}
weight: 4.4013376974108566e-05
Top 5 {'id': 625772, 'original_url': 'https://miliet.org'}
weight: 4.3353927148531034e-05
Top 6 {'id': 1152294, 'original_url': 'https://wikimediafoundation.org/'}
weight: 4.1136404671325014e-05
Top 7 {'id': 1518807, 'original_url': 'https://www.mediawiki.org/wiki/Special:MyLanguage/How_to_contribute'}
weight: 4.0199708908317586e-05
Top 8 {'id': 376674, 'original_url': 'https://foundation.wikimedia.org/wiki/Cookie_statement'}
weight: 3.9017980378992285e-05
Top 9 {'id': 376715, 'original_url': 'https://foundation.wikimedia.org/wiki/Privacy_policy'}
weight: 3.166577334836001e-05
Top 10 {'id': 1694073, 'original_url': 

In [73]:
tmp_dict = { "a" : 123}

In [74]:
len(tmp_dict)

1